In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.preprocessing import *
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import math
pd.options.display.float_format = '{:,.8f}'.format

In [2]:
application_train = pd.read_csv('D:/project/homecredit/final/application_train_ETL_v2.csv')
bureau= pd.read_csv('D:/project/homecredit/final/bureau_ETL_v2.csv',index_col=0)
bureau_balance= pd.read_csv('D:/project/homecredit/final/bureau_balance_ETL_v2.csv')
prev_creditcard= pd.read_csv('D:/project/homecredit/final/prev_creditcard_ETL_v2.csv')
pos_cash= pd.read_csv('D:/project/homecredit/final/POS_CASH_balance_ETL_v2.csv')
prev_creditcard= pd.read_csv('D:/project/homecredit/final/prev_creditcard_ETL_v2.csv')
prev_pos_instalments = pd.read_csv('D:/project/homecredit/final/prev_pos_instalments_ETL_v2.csv')
previous_application = pd.read_csv('D:/project/homecredit/final/previous_application_ETL_v2.csv',index_col=0)


In [3]:
dfs = [bureau,bureau_balance, prev_creditcard,pos_cash,prev_creditcard,prev_pos_instalments,previous_application]

for df in dfs:
    print(df.shape)

(305811, 68)
(305811, 65)
(103558, 45)
(337252, 23)
(103558, 45)
(337698, 80)
(338857, 42)


In [4]:
data = application_train 

for df in dfs:
    data = pd.merge(data, df, on="SK_ID_CURR", how="left")



# data.to_csv('D:/project/homecredit/final/ETL_all_v2.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 0 to 307510
Columns: 484 entries, SK_ID_CURR to SELLERPLACE_AREA_HIGH_rate
dtypes: float64(423), int64(41), object(20)
memory usage: 1.1+ GB


### 類別型欄位

In [5]:
data.select_dtypes(object).columns

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
       'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE',
       'CREDIT_DAY_OVERDUE_count', 'AMT_CREDIT_MAX_OVERDUE_count',
       'CNT_CREDIT_PROLONG_count', 'AMT_CREDIT_SUM_OVERDUE_count'],
      dtype='object')

In [6]:
col_to_convert =   ['CREDIT_DAY_OVERDUE_count', 'AMT_CREDIT_MAX_OVERDUE_count',
       'CNT_CREDIT_PROLONG_count', 'AMT_CREDIT_SUM_OVERDUE_count']

for col in col_to_convert:
    data[col] = data[col].replace('False', 0)

for col in col_to_convert:
    data[col] = data[col].apply(pd.to_numeric, errors='coerce')

print(data[col_to_convert].dtypes)

CREDIT_DAY_OVERDUE_count        float64
AMT_CREDIT_MAX_OVERDUE_count    float64
CNT_CREDIT_PROLONG_count        float64
AMT_CREDIT_SUM_OVERDUE_count    float64
dtype: object


In [7]:
obj_data = data.select_dtypes(object)
obj_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 0 to 307510
Data columns (total 16 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   NAME_CONTRACT_TYPE          307511 non-null  object
 1   CODE_GENDER                 307511 non-null  object
 2   FLAG_OWN_CAR                307511 non-null  object
 3   FLAG_OWN_REALTY             307511 non-null  object
 4   NAME_TYPE_SUITE             306219 non-null  object
 5   NAME_INCOME_TYPE            307511 non-null  object
 6   NAME_EDUCATION_TYPE         307511 non-null  object
 7   NAME_FAMILY_STATUS          307511 non-null  object
 8   NAME_HOUSING_TYPE           307511 non-null  object
 9   OCCUPATION_TYPE             211120 non-null  object
 10  WEEKDAY_APPR_PROCESS_START  307511 non-null  object
 11  ORGANIZATION_TYPE           307511 non-null  object
 12  FONDKAPREMONT_MODE          97216 non-null   object
 13  HOUSETYPE_MODE              1

### Onehot Encoding

In [ ]:
# obj_data= pd.get_dummies(obj_data, columns=obj_data.columns,drop_first=True)

In [ ]:
# obj_data=obj_data.drop(columns='CODE_GENDER_XNA')
# obj_data

,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,...,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes
0,0,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
307507,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
307508,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
307509,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


### 數值型欄位

In [8]:
num_data=data.select_dtypes(exclude='object')
num_data.fillna(0,inplace=True)

In [9]:
num_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 0 to 307510
Columns: 468 entries, SK_ID_CURR to SELLERPLACE_AREA_HIGH_rate
dtypes: float64(427), int64(41)
memory usage: 1.1 GB


In [10]:
num_data.shape, obj_data.shape

((307511, 468), (307511, 16))

### 合併類別型及數值型欄位

In [11]:
data = pd.concat([num_data,obj_data], axis ='columns')
data.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,AGE,JOB_TENURE,...,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,100002,1,0,"202,500.00000000","406,597.50000000","24,700.50000000","351,000.00000000",0.01880100,25,1.70000000,...,Secondary / secondary special,Single / not married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,"Stone, brick",No
1,100003,0,0,"270,000.00000000","1,293,502.50000000","35,698.50000000","1,129,500.00000000",0.00354100,45,3.30000000,...,Higher education,Married,House / apartment,Core staff,MONDAY,School,reg oper account,block of flats,Block,No
2,100004,0,0,"67,500.00000000","135,000.00000000","6,750.00000000","135,000.00000000",0.01003200,52,0.60000000,...,Secondary / secondary special,Single / not married,House / apartment,Laborers,MONDAY,Government,NaN,NaN,NaN,NaN
3,100006,0,0,"135,000.00000000","312,682.50000000","29,686.50000000","297,000.00000000",0.00801900,52,8.30000000,...,Secondary / secondary special,Civil marriage,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,NaN,NaN,NaN,NaN
4,100007,0,0,"121,500.00000000","513,000.00000000","21,865.50000000","513,000.00000000",0.02866300,54,8.30000000,...,Secondary / secondary special,Single / not married,House / apartment,Core staff,THURSDAY,Religion,NaN,NaN,NaN,NaN


In [ ]:
data.to_csv('D:/project/homecredit/final/ETL_ALL_not_encoding_v2.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 0 to 307510
Columns: 591 entries, SK_ID_CURR to EMERGENCYSTATE_MODE_Yes
dtypes: float64(427), int64(41), uint8(123)
memory usage: 1.1 GB
